In [4]:
%matplotlib inline
%load_ext heat

In [18]:
from IPython.display import Image
from copy import copy,deepcopy
from numpy import pi
import numpy as np
from qutip import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit, Gate
import time
from itertools import combinations


def user_gate1():
# S gate
    mat = np.array([[1.,0],[0., 1.j]])
    return Qobj(mat, dims=[[2], [2]])

def user_gate2():
# T gate
    mat = np.array([[1.,0],[0., 0.707+0.707j]])
    return Qobj(mat, dims=[[2], [2]])

In [19]:
def qu(gnum,n): # pick the unitary from the gate set #error
    if gnum<2*n+1:
        #print('a'+str(int((gnum-1)/n)))
        if gnum%n==0:
            g_t= Gate("SNOT", int((gnum-1)/n))
        else: 
            g_t= Gate("T",int((gnum-1)/n))
    elif gnum<2*n+n+1:
        g_t= Gate()
    return g_t

In [20]:
def quc(x,n): # generate the given quantum circuit
    qc= QubitCircuit(n)
    qc.user_gates = {"S": user_gate1,"T": user_gate2}
    for i in range(len(x)):
        if x[i]!=0:
            qc.add_gate(qu(x[i], n))
    prop = qc.propagators()
    return gate_sequence_product(prop).full()
    

In [21]:
#may be faster to apply inverse i.e. loop over numbers gmax**r in base gmax
# def list_circuits(gmax,r):# list the circuits
#     def list_c(*arrays): 
#         grid = np.meshgrid(*arrays)        
#         coord_list = [entry.ravel() for entry in grid]
#         points = np.vstack(coord_list).T
#         return points
#     aa = np.arange(gmax)
#     return list_c(*r*[np.arange(gmax)])

# below is time inefficient but memory efficient
# not optimal 28.8 vs 15.7

def base_conv(cirnum,gmax,r): 
    xx= np.zeros(r)
    num=np.copy(cirnum)
    for i in range(r): 
        xx[i]= num%gmax
        num//=gmax
    return xx



In [22]:
def bi(ii,n):
    bix = np.zeros(n)
    for i in range(n): 
        bix[i]= ii%2
        ii//=2
    return(bix)

def list_povm(n): 
    l_povms=[]
    for i in range(2**n):
        pv = np.zeros(2**n)
        b = bi(i,n)
        pv[0]= 1
        for j in range(len(b)): 
            if b[j]==1:
                pv[2**j:2**(j+1)]= pv[:2**j]
        l_povms.append(np.diag(pv))
    return(l_povms)


# below is time inefficient but memory efficient
#removed since, it is less optimal 4.67s vs 3.54s


def gene_povm(gnp,n): 
    pv = np.zeros(2**n)
    b = gnp
    pv[0]= 1
    for j in range(len(b)): 
        if b[j]==1:
            pv[2**j:2**(j+1)]= pv[:2**j]
    return(np.diag(pv)) 

            
                
                

In [23]:
def trqrho(q,rho):
    trq= 0
    trq+= np.dot(np.diag(q),np.diag(rho))
    return(trq)

def tr(q): 
    return(np.sum(q))


In [24]:
# All the inputs are in this box 

# system specific data
n = 3 # number of qubits
gmax= 1+3*n+2*n # number of unique gates in the universal gate set




In [25]:
# process specific
r = 4 # max complexity
eta = 0.9 # success rate? 
# define rho below this
x = np.random.rand(2**n,2**n)
rho= x@x.transpose()
rho = rho/np.trace(rho)


In [27]:
ndg_list=[]
tt=0
for cnum in range(0,gmax**r):
    bc=base_conv(cnum,gmax,r)
    if np.all(bc[bc !=0]-bc[:len(bc[bc !=0])]==0):
        #print(cnum)
        #print(bc)
        ndg_list = np.append(ndg_list,cnum)
        #print(ndg_list)
print(ndg_list)

[0.0000e+00 1.0000e+00 2.0000e+00 ... 6.5533e+04 6.5534e+04 6.5535e+04]


In [14]:
circi=0
povmi=np.zeros(n)
H=2**n
for cnum in np.nditer(ndg_list):
    #print(cnum)
    #print(base_conv(cnum,gmax,r))
    bc = base_conv(cnum,gmax,r)
    U = quc(bc,n)
    #print(U)
    sigma = U@rho@U.conj() # use of einsum may make it faster? need to make this step faster? sparce?
    counter = 0
    for qbits in range(n+1): 
        for pv in combinations(np.arange(n),n-qbits):
            pov = np.zeros(n)
            for iden in  range(len(pv)):
                pov[pv[iden]]= 1
            povm = gene_povm(pov,n)
            if trqrho(sigma,povm)>=eta: 
                counter+=1
                if tr(povm)<H:
                    circi = cnum
                    povmi = pov
                    H = tr(povm)
        if counter==0:
            break
        else: 
            counter=0

print("effective entropy for eta="+str(eta)+" and r="+str(r) +" is "+ str(np.log(H)/np.log(2)))    

NameError: name 'np' is not defined

In [34]:
print(povmi)#873
print(circi)
U = quc(base_conv(circi,gmax,r),n)
print(np.diag(U@rho@U.conj()))
print(np.sum(np.diag(U@rho@U.conj())))

[0. 1. 1.]
2714.0
[0.20188347+0.j 0.01262798+0.j 0.19867605+0.j 0.0068577 +0.j
 0.22558329+0.j 0.02378693+0.j 0.2994622 +0.j 0.03112239+0.j]
(0.9999999999999997+0j)


In [ ]:
#data from before
# [[1. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 1. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 0.]]
# 54
# [0.37804113+0.j 0.44065229+0.j 0.03834387+0.j 0.04410131+0.j
#  0.02560767+0.j 0.01531818+0.j 0.01909968+0.j 0.03883588+0.j]
# (0.9999999999999996+0j)


In [44]:
# Below is testing rough ideas 

In [114]:
counter = 0
for qbits in range(n+1): 
    for pv in combinations(np.arange(n),n-qbits):
        pov = np.zeros(n)
        for iden in  range(len(pv)):
            pov[pv[iden]]= 1
        povm = gene_povm(pov,n)
        if trqrho(sigma,povm)>=eta: 
            counter+=1
            if tr(povm)<H:
                circi = cnum
                povmi = pov
                H = tr(povm)
    if counter==0:
        break
    else: 
        counter=0

        
    

In [170]:
a  = np.zeros(5)
a[3]=2
a[1]=1


In [ ]:
if np.all(bc[bc !=0]-bc[:len(bc[bc !=0])]==0):


In [171]:
 b =a[a !=0]
if np.all(a[a !=0]-a[:len(a[a !=0])]==0):
    print("yes")
else:
    print("no")

no


In [ ]:
def base_inv
def degeneracy_check(x,gmax):
    y = x[x !=0]
    
    

In [64]:
qc= QubitCircuit(n)
qc.add_gate("SNOT", 1)
qc.add_gate("CNOT",1,2)

In [51]:
a = np.random.rand(8,8)
b = np.random.rand(8,8)
c = a@b@a.conj()
np.log(2.7)

0.9932517730102834

In [65]:
gnum=11
(gnum-3*n)%n, (gnum-3*n+1)%n

(2, 0)

In [92]:
a = combinations(np.arange(5),2)
#print(list(a)[0])
for i in a:
    for j in range(len(i)): 
        print(i[j])

0
1
0
2
0
3
0
4
1
2
1
3
1
4
2
3
2
4
3
4


In [109]:
print(5-np.arange(5))

[5 4 3 2 1]


5%2


In [22]:
props = qc.propagators()

In [46]:
U= gate_sequence_product(props).full()
U.conj()
# Q = np.random.rand(2**n,2**n)
# np.trace(U*Q)
    


array([[ 0.70710678-0.j,  0.        -0.j,  0.70710678-0.j,
         0.        -0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j],
       [ 0.        -0.j,  0.70710678-0.j,  0.        -0.j,
         0.70710678-0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j],
       [ 0.70710678-0.j,  0.        -0.j, -0.70710678-0.j,
         0.        -0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j],
       [ 0.        -0.j,  0.70710678-0.j,  0.        -0.j,
        -0.70710678-0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j],
       [ 0.        -0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.70710678-0.j,  0.        -0.j,
         0.70710678-0.j,  0.        -0.j],
       [ 0.        -0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j,  0.70710678-0.j,
         0.        -0.j,  0.70710678-0.j],
       [ 0.        -0.j,  0.      

In [10]:
props = qc1.propagators()
props[0]

NameError: name 'qc1' is not defined

In [11]:
for i in range(10,13):
    print(i,(i-3*n)%n, (i-3*n+1)%n)

10 1 2
11 2 0
12 0 1


In [112]:
for i in range(5+1):
    print(i)

0
1
2
3
4
5


In [13]:
a


NameError: name 'a' is not defined

In [15]:
a = np.arange(50)  # fake data
print(len(list_c(*r*[a])))


NameError: name 'list_c' is not defined

In [46]:
pv = np.random.rand(2**5)
j = 3
pv[2**j:2**(j+1)]= pv[:2**j]
print(pv[:2**5])

[0.49171691 0.98149052 0.31583407 0.50734355 0.71633766 0.6033593
 0.72866585 0.74982201 0.49171691 0.98149052 0.31583407 0.50734355
 0.71633766 0.6033593  0.72866585 0.74982201 0.57805718 0.4293471
 0.21136173 0.95261813 0.1583954  0.48094306 0.41077553 0.58631585
 0.70192571 0.74992607 0.46552065 0.14574783 0.05704509 0.02934502
 0.37451038 0.40412242]


In [111]:
range(5,1)

range(5, 1)

In [28]:
a = gmax**r
print(a)

65536
